### Ali Ataollahi - 810199461

## Question 1

In [1]:
import re

def custom_tokenizer(text):
  pattern = r'\b\w+\b'
  tokens = re.findall(pattern, text)
  return tokens

### Section 1
Which type of tokenizer does the code above suggest? Is it character-based, subword-based, or word-based? 

It is word-based tokenizer. The regular expression pattern r'\\b\\w+\\b' breaks down as follows:

+ \\b matches a word boundary (the position between a word character and a non-word character)
+ \\w+ matches one or more word characters (alphanumeric characters and underscore)
+ The second \\b matches another word boundary

So this regular expression will match sequences of word characters that are delimited by word boundaries on both sides. In other words, it will match whole words. This is different from a character-based tokenizer, which would split the text into individual characters. And it's also different from a subword-based tokenizer, which would split words into smaller meaningful subword units like prefixes, suffixes, and stems.

Describe the problems of this type of tokenizer with examples.

One limitation of this approach is that it struggles with words containing punctuation, resulting in tokens that might be difficult to comprehend. Take the word "Can't," for instance, which could be tokenized as "Can" and "'t" instead of the more accurate "Ca" and "n't." Word-based tokenizers struggle with misspelled words or words that are concatenated without spaces (e.g., "cheezburger" or "iloveyou").
Additionally, this method fails to recognize the similarity between words sharing the same root. Consider the words "Run" and "Running"; the technique may not capture their linguistic connection. Another drawback of word-based tokenization is the creation of an extensive vocabulary. A fundamental cause of this challenge lies in the previously mentioned issue: the failure to grasp linguistic similarities. Consequently, the method stores variations of the same root word multiple times, contributing to a bloated vocabulary.

### Section 2

In [2]:
str = "Just received my M.Sc. diploma today, on 2024/02/10! Excited to embark on this new journey of knowledge and discovery. #MScGraduate #EducationMatters."
print(custom_tokenizer(str))

['Just', 'received', 'my', 'M', 'Sc', 'diploma', 'today', 'on', '2024', '02', '10', 'Excited', 'to', 'embark', 'on', 'this', 'new', 'journey', 'of', 'knowledge', 'and', 'discovery', 'MScGraduate', 'EducationMatters']


+ The abbreviation "M.Sc." is incorrectly split into two separate tokens "M" and "Sc" instead of being treated as a single token. Because it has meaning when they come together.
+ The date "2024/02/10" is split into three separate tokens "2024", "02", and "10" instead of being recognized as a single date entity.
+ The hashtags "#MScGraduate" and "#EducationMatters" are split into separate tokens, losing the meaning and structure of the hashtags.


### Section 3

In [3]:
import re

def custom_tokenizer(text):
    pattern = pattern = r'\b(?:[A-Z]\.[A-Za-z]+|\d{4}/\d{2}/\d{2}|\d+(?:/\d+)?|[A-Za-z]+)\b'
    # r'\b(?:\w+\.?\w*|\d+(?:/\d+)*(?:\.\d+)?)\b'


    tokens = re.findall(pattern, text)

    return tokens

text = "Just received my M.Sc. diploma today, on 2024/02/10! Excited to embark on this new journey of knowledge and discovery. #MScGraduate #EducationMatters."
tokens = custom_tokenizer(text)
print(tokens)

['Just', 'received', 'my', 'M.Sc', 'diploma', 'today', 'on', '2024/02/10', 'Excited', 'to', 'embark', 'on', 'this', 'new', 'journey', 'of', 'knowledge', 'and', 'discovery', 'MScGraduate', 'EducationMatters']


I use another pattern and some of problems are resolved. But another problems remaining. Resolving this problem necessitates more sophisticated methods and advancements in the approach.

## Question 2

### Section 1

Tokenizer used in each of BERT and GPT language models, which of the letter-based, sub-word-based or word-based types? Justify the reason for choosing this type of tokenizers in large language models.

- BERT uses a sub-word-based tokenizer, specifically the WordPiece tokenizer. WordPiece is a type of subword tokenization algorithm that breaks down words into smaller, meaningful subword units (like "un##case" and "##load"). 
- The GPT language models (GPT-1, GPT-2, and GPT-3) use a byte-level Byte-Pair Encoding (BPE) tokenizer, which is also a sub-word-based tokenization approach.

The main reason for using sub-word-based tokenizers (like WordPiece in BERT and Byte-Pair Encoding in GPT) in large language models like BERT and GPT is to effectively handle out-of-vocabulary (OOV) words. By breaking down words into smaller, meaningful subword units, these tokenizers can represent even rare or unseen words by combining known subword tokens. This helps mitigate the OOV issue, which can significantly impact the performance of language models that use a fixed vocabulary.

### Section 2

BERT and GPT language models have used two different algorithms to implement their custom tokenizers. Name these two algorithms and briefly describe their differences.

#### BERT tokenizer algorithm
Tokenizer used in BERT is based on the WordPiece algorithm, which is a type of subword tokenization algorithm. The WordPiece algorithm works by first building a vocabulary of the most frequent subword units (starting from individual characters and use $\frac{\text{Pair Frequency}}{\text{First Element Frequency} \times \text{Second Element Frequency}}$ for select most frequent subword units) from the training data. It then iteratively combines the most frequent pairs of tokens to create new tokens, until a desired vocabulary size is reached. 

#### GPT tokenizer algorithm

GPT employs the Byte-Pair Encoding (BPE) Method, a dynamic algorithm that commences by analyzing the distinct set of words within the given dataset. Subsequently, it constructs a vocabulary incorporating the symbols necessary to form these words. The algorithm then initiates the learning process by identifying and merging the most frequent consecutive pairs of symbols. The resulting merged subwords are systematically incorporated into the expanding vocabulary. This iterative merging process continues, gradually enlarging the size of the learned subwords and enhancing the model's linguistic capabilities. The dynamic nature of BPE ensures an adaptive and effective approach to language representation within GPT.

#### Differences

The WordPiece and BPE algorithms differ in their starting point, merging criteria, tokenization unit, and vocabulary size control. WordPiece begins with individual characters and builds subword units, merging the most frequent token pairs, and tokenizes text into subword units with direct control over vocabulary size. The Main difference between these two algorithms lies in their utilization of the scoring function. WordPiece takes into account the frequency of individual elements, whereas the BPE method exclusively focuses on the frequency of the pairs themselves. On the other hand, BPE starts with individual characters but merges them into larger subword units, merging the most frequent pairs of consecutive characters, and tokenizing text into subword units composed of one or more characters. Despite these distinctions, both algorithms share the common goal of efficiently handling out-of-vocabulary words by breaking them down into smaller, meaningful subword units, enabling language models to represent a vast vocabulary with a limited number of tokens.

### Section 3 

Develop a basic implementation of these algorithms.

#### WordPiece (for BERT)

In [1]:
import re
from collections import Counter
from typing import Dict, List, Tuple

def corpus_to_words(corpus: str) -> List[str]:
    """Split the corpus into words."""
    return re.findall(r"\w+|[.,!?;]", corpus)

def create_freq_map(words: List[str]) -> Dict[str, int]:
    """Create a frequency map of words."""
    return dict(Counter(words))

def create_initial_vocab(words: List[str]) -> set:
    """Create the initial vocabulary (alphabet)."""
    vocab = set()
    for word in words:
        vocab.update(set(word))
    return vocab

def word_to_chars(word: str) -> List[str]:
    """Convert a word to a list of characters."""
    return list(word)

def word_piece_pairs_score(splits: Dict[str, List[str]], word_freqs: Dict[str, int]) -> Dict[Tuple[str, str], int]:
    """Calculate the score for each pair of characters."""
    pair_scores = {}
    for word, chars in splits.items():
        for i in range(len(chars) - 1):
            pair = ''.join(chars[i:i+2])
            pair_scores[pair] = pair_scores.get(pair, 0) + word_freqs[word]
    return pair_scores

def merge_pair(splits: Dict[str, List[str]], pair: Tuple[str, str]) -> None:
    """Merge the best pair of characters."""
    for word, chars in splits.items():
        new_chars = []
        i = 0
        while i < len(chars):
            if ''.join(chars[i:i+2]) == pair:
                new_chars.append(pair[0] + pair[1])
                i += 2
            else:
                new_chars.append(chars[i])
                i += 1
        splits[word] = new_chars

def train_word_piece(corpus: str, vocab_size: int) -> Tuple[List[str], Dict[Tuple[str, str], str]]:
    """Train the WordPiece model."""
    words = corpus_to_words(corpus)
    word_freqs = create_freq_map(words)
    vocab = create_initial_vocab(words)
    splits = {word: word_to_chars(word) for word in word_freqs.keys()}
    merges = {}

    while len(vocab) < vocab_size:
        scores = word_piece_pairs_score(splits, word_freqs)
        best_pair = max(scores, key=scores.get)
        merge_pair(splits, best_pair)
        vocab.add(best_pair[0] + best_pair[1])
        merges[best_pair] = best_pair[0] + best_pair[1]

    return list(vocab), merges

def word_piece_largest_word(word: str, vocab: List[str]) -> List[str]:
    """Find the largest subwords in the vocabulary."""
    word_pieces = []
    i = 0
    while i < len(word):
        for j in range(len(word), i, -1):
            subword = word[i:j]
            if subword in vocab:
                word_pieces.append(subword)
                i = j
                break
    return word_pieces

def tokenize(text: str, vocab: List[str], merges: Dict[Tuple[str, str], str]) -> List[str]:
    """Tokenize the text using the learned WordPiece model."""
    words = corpus_to_words(text)
    tokens = []
    for word in words:
        word_pieces = word_piece_largest_word(word, vocab)
        tokens.extend(word_pieces)
    return tokens

text_to_tokenize = "This darkness is absolutely killing! If we ever take this trip again, it must be about the time of the New Moon"

file_path = "data/All_Around_the_Moon.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    text_corpus = file.read()

vocab, merges = train_word_piece(text_corpus, 300)
tokens = tokenize(text_to_tokenize, vocab, merges)
print(tokens)

['Th', 'is', 'da', 'r', 'kn', 'es', 's', 'is', 'ab', 'so', 'lu', 'te', 'ly', 'ki', 'll', 'in', 'g', '!', 'If', 'we', 'ev', 'er', 'ta', 'ke', 'th', 'is', 'tr', 'i', 'p', 'ag', 'ai', 'n', ',', 'it', 'mu', 'st', 'be', 'ab', 'ou', 't', 'th', 'e', 'ti', 'me', 'of', 'th', 'e', 'N', 'e', 'w', 'Mo', 'on']


#### BPE (for GPT)

In [2]:
from collections import defaultdict, Counter
from typing import List, Dict, Tuple

def bpe_pairs_score(splits: Dict[str, List], words_freq: Counter) -> Dict[Tuple[str, str], int]:
    """
    Scoring function for BPE algorithm. The score of each pair is simply the
    frequency of that pair in the corpus.
    """

    pair_freq = defaultdict(int)

    for word, split in splits.items():
        for i in range(len(split) - 1):
            pair_freq[(split[i], split[i + 1])] += words_freq[word]

    return pair_freq

def corpus_to_words(corpus: str) -> List[str]:
    return corpus.split()

def word_piece_largest_word(word: str, vocab: List[str]) -> List[str]:
    return [word]

def merge_pair(splits: Dict[str, List], pair: Tuple[str, str], vocab: List[str]) -> Dict[str, List]:
    """
    Merge the most frequent pair in the splits.
    """
    merged_pair = "".join(pair)
    new_splits = {}

    for word, split in splits.items():
        new_split = []
        i = 0
        while i < len(split):
            if i < len(split) - 1 and (split[i], split[i + 1]) == pair:
                new_split.append(merged_pair)
                i += 2
            else:
                new_split.append(split[i])
                i += 1

        new_splits[word] = new_split

    return new_splits

def train_bpe(corpus: str, vocab_size: int) -> Tuple[List[str], Dict[Tuple[str, str], str]]:
    """
    Train BPE on the given corpus and return the vocabulary and merges.
    """
    words_freq = Counter(corpus_to_words(corpus))
    vocab = list(words_freq.keys())

    splits = {word: list(word) for word in vocab}

    for _ in range(vocab_size):
        pair_freq = bpe_pairs_score(splits, words_freq)
        if not pair_freq:
            break

        best_pair = max(pair_freq, key=pair_freq.get)
        vocab.append("".join(best_pair))

        splits = merge_pair(splits, best_pair, vocab)

    merges = {(pair[0], pair[1]): merged for pair, merged in zip(pair_freq.keys(), vocab[vocab_size:])}

    return vocab, merges

text_to_tokenize = "This darkness is absolutely killing! If we ever take this trip again, it must be about the time of the New Moon"

file_path = "data/All_Around_the_Moon.txt" 
with open(file_path, 'r', encoding='utf-8') as file:
    text_corpus = file.read()

vocab, merges = train_bpe(text_corpus, 300)
tokens = tokenize(text_to_tokenize, vocab, merges)
print("Tokens:", tokens)

Tokens: ['This', 'darkness', 'is', 'absolutely', 'killing!', 'If', 'we', 'ever', 'take', 'this', 'trip', 'again,', 'it', 'must', 'be', 'about', 'the', 'time', 'of', 'the', 'New', 'Moon']


## Question 3

### Section 1 - Train the n-gram language model using the pre-processed text data

We load the data file, create a new tokenizer instance with the Byte-Pair Encoding (BPE) model, set up the normalizer and pre-tokenizer, define special tokens, create a BPE trainer, train the tokenizer on the data file using the BPE trainer, and set up the post-processor for the tokenizer.

In [3]:
import os

from random import choices

from tokenizers import models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

file_path = open("./data/Tarzan.txt", encoding="utf8")

file_content = file_path.read()

tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase()]
)

tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

special_tokens = ["[SEP]", "[MASK]", "[UNK]", "[PAD]", "[CLS]"]

token_trainer = trainers.BpeTrainer(special_tokens=special_tokens)

tokenizer.train(["./data/Tarzan.txt"], trainer=token_trainer)

tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:1",  
    pair="[CLS]:0 $A:0 [SEP]:1 $B:1 [SEP]:2",
    special_tokens=[("[CLS]", tokenizer.token_to_id("[CLS]")), ("[SEP]", tokenizer.token_to_id("[SEP]"))],
)

### Section 2 - Bigram Preparation and Language Model Training

In [4]:
from collections import Counter
from typing import List, Dict, Tuple

def create_n_grams(text: str, n: int, tk: Tokenizer) -> list[tuple[str]]:
    tokens = tk.encode(text).tokens
    result_n_grams = []
    idx_range = range(len(tokens) - n + 1) if n > 0 else range(len(tokens) - n)
    for i in idx_range:
        result_n_grams = result_n_grams + [tuple(tokens[i:i+n])]
    return result_n_grams


#### Data sparsity problem
It refers to the problem where some n-grams in the test data may not be present in the training data, leading to zero probabilities and inability to generate those n-grams. To handle data sparsity, you can use techniques like smoothing (e.g., add-one smoothing, Good-Turing smoothing) or backoff models, which use lower-order n-grams when higher-order n-grams are not found in the training data.

To address the data sparsity issue in n-gram language models, where some n-grams are rarely or never observed in the training data, leading to zero or inaccurate probabilities, several techniques can be employed: smoothing methods like Laplace smoothing, Good-Turing smoothing, and Kneser-Ney smoothing add small constants to n-gram counts to avoid zero probabilities; backoff models use lower-order n-gram probabilities when higher-order n-grams are unseen; class-based n-grams cluster words into classes and calculate probabilities over classes instead of individual words; interpolation combines probabilities from different n-gram models using weighted averages; higher-order n-grams capture longer contexts but increase model complexity; and neural language models like LSTMs and Transformers can learn distributed representations and mitigate data sparsity more effectively than traditional n-gram models, although at the cost of increased computational requirements.

### Section 3

In [5]:
from collections import Counter
from typing import List, Dict, Tuple

def create_n_grams(text: str, n: int, tk: Tokenizer) -> list[tuple[str]]:
    tokens = tk.encode(text).tokens
    result_n_grams = []
    idx_range = range(len(tokens) - n + 1) if n > 0 else range(len(tokens) - n)
    for i in idx_range:
        result_n_grams = result_n_grams + [tuple(tokens[i:i+n])]
    return result_n_grams

def calculate_and_create_n_gram_probability(text: str, n: int, tk: Tokenizer) -> Dict[Tuple[str], float]:
    tokens = tk.encode(text).tokens
    result_n_grams = []
    idx_range = range(len(tokens) - n + 1) if n > 0 else range(len(tokens) - n)
    for i in idx_range:
        result_n_grams.append(tuple(tokens[i:i+n]))
    big_sentences = Counter(create_n_grams(text, n, tokenizer))
    small_sentences = Counter(create_n_grams(text, n - 1, tokenizer))
    result = {}
    for big_sentence, big_sentence_count in big_sentences.items():
        small_sentence_count = small_sentences[big_sentence[:-1]]
        result[big_sentence] = big_sentence_count / small_sentence_count
    return result

In [6]:
def predict_next_word(prev_seq: list[str], n_gram: dict[tuple[str], int]) -> str | None:
    matched_seqs: list[tuple[str]] = []
    seq_probs: list[int] = []
    prev_seq = tuple(prev_seq)
    for words_seq, probability in n_gram.items():
        if prev_seq == words_seq[:-1]:
            matched_seqs += [words_seq]
            seq_probs += [probability]
    if not matched_seqs:
        return None
    return choices(matched_seqs, seq_probs)[0][-1] # Select the last word of the chosen n-gram

def tokenize(
    init_seq: str,
    n: int,
    trained_n_grams: list[dict[tuple[str], int]],
    tk: Tokenizer) -> list[str]:
    n_tokens = 10
    result = tk.encode(init_seq).tokens[:-1]  # Tokenize and remove the end-of-sequence special token
    current_n = n
    for i in range(n_tokens):
        next_token = None
        while next_token is None and current_n > 0:
            next_token = predict_next_word(result[-(current_n - 1):], trained_n_grams[current_n - 1])
            current_n -= 1
        if next_token is None:
            break
        result.append(next_token)
        current_n = n  
    return result

def create_n_grams_set(text: str, n: int, tk: Tokenizer) -> list[dict[tuple[str], int]]:
    result = [None] * n
    for i in range(1, n + 1):
        result[i - 1] = calculate_and_create_n_gram_probability(text, i, tk)
    return result

trained_n_grams_2 = create_n_grams_set(file_content, 2, tokenizer)

init_seq_1 = "Knowing well the windings of the trail he"
print(tokenize(init_seq_1, 2, trained_n_grams_2, tokenizer))

init_seq_2 = "For half a day he lolled on the huge back and"
print(tokenize(init_seq_2, 2, trained_n_grams_2, tokenizer))

['[CLS]', 'knowing', 'well', 'the', 'windings', 'of', 'the', 'trail', 'he', 'had', 'been', 'excited', 'himself', 'responsible', ',', 'riding', 'back', 'into', 'the']
['[CLS]', 'for', 'half', 'a', 'day', 'he', 'lolled', 'on', 'the', 'huge', 'back', 'and', 'more', 'than', 'the', 'center', 'of', 'nimmr', 'in', 'his', 'only', 'protection']


### Section 4

In [7]:
trained_n_grams_3 = create_n_grams_set(file_content, 3, tokenizer)

print(tokenize(init_seq_1, 3, trained_n_grams_3, tokenizer))
print(tokenize(init_seq_2, 3, trained_n_grams_3, tokenizer))

trained_n_grams_5 = create_n_grams_set(file_content, 5, tokenizer)

print(tokenize(init_seq_1, 5, trained_n_grams_5, tokenizer))
print(tokenize(init_seq_2, 5, trained_n_grams_5, tokenizer))

['[CLS]', 'knowing', 'well', 'the', 'windings', 'of', 'the', 'trail', 'he', 'took', 'with', 'seven', 'great', 'lions', 'watching', 'his', 'slow', 'ascent', 'toward']
['[CLS]', 'for', 'half', 'a', 'day', 'he', 'lolled', 'on', 'the', 'huge', 'back', 'and', 'overtake', 'you', 'without', 'much', 'loss', 'of', 'time', '.', '"', 'the']


### Section 5

Is it possible to increase the parameter n in the n-gram language model without limits? Why?

No, it is not practical to increase the parameter 'n' without limitations. The primary challenge stems from the escalating computational demands for training such a model. Furthermore, a more significant concern arises from the heightened data sparsity. As we aspire to model larger combinations of words, the probability of encountering these extensive combinations diminishes substantially. This results in the model learning only a limited set of word combinations. For instance, when considering the initial phrase "Exploring Uncharted Territories," the corpus contains fewer instances of such combinations, making it more challenging for the model to accurately predict the subsequent words in the sequence.

## Question 4

### Section 1 - Implementing a Test Function for N-Gram Language Model

In [ ]:
import nltk
import numpy as np
import pandas as pd
from nltk import ngrams
from nltk.probability import FreqDist
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve

nltk.download('punkt')

In [ ]:
# Step 1: Load the data
data = pd.read_csv('data/google_play_store_apps_reviews.csv')

# Step 2: Split the data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
# Step 3: Build the n-gram Language Model
def get_ngrams(text, n):
    tokens = nltk.word_tokenize(text)
    return list(ngrams(tokens, n))

def train_ngram(data, n):
    positive_ngrams = []
    negative_ngrams = []

    for index, row in data.iterrows():
        grams = get_ngrams(row['review'], n)
        if row['polarity'] == 1:
            positive_ngrams.extend(grams)
        elif row['polarity'] == 0:
            negative_ngrams.extend(grams)

    positive_freq = FreqDist(positive_ngrams)
    negative_freq = FreqDist(negative_ngrams)

    return positive_freq, negative_freq

# Step 4: Train the Model
n = 2  # Change to the desired n-gram size
positive_freq, negative_freq = train_ngram(train_data, n)

In [ ]:
# Step 5: test the n-gram
def test_ngram(data, positive_freq, negative_freq, n):
    pred_labels = []

    for index, row in data.iterrows():
        grams = get_ngrams(row['review'], n)
        positive_prob = 1
        negative_prob = 1

        for gram in grams:
            positive_prob *= positive_freq[gram] + 1
            negative_prob *= negative_freq[gram] + 1

        positive_prob = positive_prob * sum(negative_freq.values()) / sum(positive_freq.values())

        if positive_prob > negative_prob:
            pred_labels.append(1)
        else:
            pred_labels.append(0)

    return pred_labels


### Section 2 - Performance Evaluation and Reporting for N-Gram Language Models

In [ ]:
# Step 6: Evaluate the model on the test set
n = 2  
test_labels = test_ngram(test_data, positive_freq, negative_freq, n)

true_labels = test_data['polarity'].tolist()

accuracy = accuracy_score(true_labels, test_labels)
precision = precision_score(true_labels, test_labels)
recall = recall_score(true_labels, test_labels)

print("Accuracy Score:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

The model has an accuracy of approximately 79.89%, indicating overall correctness. It exhibits high precision (94.74%), signifying accurate positive predictions, but a low recall (33.96%), suggesting it may miss a notable portion of actual positive instances. The balance between precision and recall depends on the task requirements.